In [408]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import numpy as np


In [38]:

driver = webdriver.Chrome()
driver.get('https://www.yvr.ca/en/passengers/flights/arriving-flights')

html = driver.page_source
soup = BeautifulSoup(html)

# check out the docs for the kinds of things you can do with 'find_all'
# this (untested) snippet should find tags with a specific class ID
# see: http://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-by-css-class
for tag in soup.find_all("a", class_="my_class"):
    print(tag.text)

In [278]:
#DONE!!!
flight_nums = soup.findAll("td", {"class": "yvr-table__cell yvr-flights__flightNumber notranslate"})
airline = soup.findAll("td", {"class": "hidden-until--md yvr-table__cell yvr-table__cell--fade-out yvr-table__cell--nowrap notranslate"})
origins = soup.findAll("td", {"class": "yvr-table__cell yvr-table__cell--fade-out yvr-table__cell--nowrap notranslate"})
raw_rows = soup.tbody.findAll('tr', {'class':'yvr-flights__row'})
raw_rows.pop(0)

flight_ids = list(map(lambda x: x['id'],raw_rows))
flight_nums = list(map(lambda x: x.text, flight_nums))
airline = list(map(lambda x: x.text, airline))
origins = list(map(lambda x: x.text, origins))

In [335]:
# TO TROUBLESHOOT

scheduled_arr = soup.findAll(['div','time'], {"class": ["yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--arrived",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--delayed",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--cancelled",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--early",
                                           "yvr-flights__label yvr-flights__scheduled-label yvr-flights__scheduled-label--str_on_time notranslate"]})
#estimated_arr = soup.findAll("td.time", {"class": ""})
#status = soup.findAll("td", {"class": "yvr-flights__status yvr-flights__status--arrived"})

In [384]:
YVR_arrivals = pd.DataFrame({"id": flight_ids, "flight_number": flight_nums, "airline": airline, "origin":origins})

In [385]:
YVR_arrivals.loc[2:4,:]

,id,flight_number,airline,origin
2,299767237,UA5754,United Airlines,San Francisco
3,299766105,WS2061,WestJet,Los Cabos
4,299765117,AC278,Air Canada,Whitehorse


In [336]:
scheduled_arr = list(map(lambda x: x['datetime'], scheduled_arr))
times = pd.DataFrame({"scheduled":scheduled_arr})

In [330]:
times.shape

(87, 1)

In [331]:
YVR_arrivals.shape

(88, 3)

In [333]:
pd.concat([YVR_arrivals,times], axis=1)

,flight_number,airline,origin,scheduled
0,WS717,WestJet,Toronto,2020-02-09T18:23:00-08:00
1,AS2314,Alaska Airlines,Seattle,2020-02-09T18:33:00-08:00
2,UA5754,United Airlines,San Francisco,2020-02-09T18:36:00-08:00
3,WS2061,WestJet,Los Cabos,2020-02-09T18:52:00-08:00
4,AC278,Air Canada,Whitehorse,2020-02-09T19:00:00-08:00
...,...,...,...,...
83,CX865,Cathay Pacific,JFK New York,2020-02-09T23:50:00-08:00
84,AC1739,Air Canada,Cancun,2020-02-09T23:52:00-08:00
85,WS2027,WestJet,PuertoVallarta,2020-02-09T23:52:00-08:00
86,WS1791,WestJet,Las Vegas,2020-02-09T23:59:00-08:00


In [328]:
### GET ESTIMATED



In [337]:
raw_rows[1]

<tr class="yvr-flights__row yvr-flights__row--earlier yvr-flights__row--hidden yvr-flights__row--arrived" id="299762601">
<td>
<time class="yvr-flights__label yvr-flights__scheduled-label yvr-flights__scheduled-label--arrived notranslate" datetime="2020-02-09T18:23:00-08:00">
					18:23
				</time>
</td>
<td class="yvr-flights__table-cell--revised notranslate">
<time class="yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--arrived" datetime="2020-02-09T18:23:00-08:00">
							18:54
						</time>
</td>
<td class="yvr-table__cell yvr-flights__flightNumber notranslate">WS717</td>
<td class="hidden-until--md yvr-table__cell yvr-table__cell--fade-out yvr-table__cell--nowrap notranslate">WestJet</td>
<td class="yvr-table__cell yvr-table__cell--fade-out yvr-table__cell--nowrap notranslate">Toronto</td>
<td class="hidden-until--md yvr-table__cell yvr-table__cell--nowrap notranslate">
Main				
			</td>
<td class="hidden-until--md yvr-table__cell yvr-table__cell--nowrap n

In [355]:
#soup.tbody

In [383]:
raw_rows = soup.tbody.findAll('tr', {'class':'yvr-flights__row'})
raw_rows.pop(0)
flight_ids = list(map(lambda x: x['id'],raw_rows))

In [386]:
len(raw_rows)

88

In [390]:
raw_rows[1].findAll('time')[0]['datetime']

'2020-02-09T18:33:00-08:00'

In [394]:
pairs = list(map(lambda x: x.findAll('time'), raw_rows))

In [396]:
len(pairs)

88

In [402]:
pairs[0][0]['datetime']

'2020-02-09T18:23:00-08:00'

In [399]:
a.append(1)
a

[1, 1]

In [409]:
sched = []
est = []
for i in pairs:
    if len(i) == 2:
        sched.append(i[0]['datetime'])
        est.append(i[1]['datetime'])
        
    if len(i) == 1:
        sched.append(i[0]['datetime'])
        est.append(np.nan)

In [410]:
len(sched)

88

In [411]:
len(est)

88

In [232]:
len(soup.findAll(['td','time'], {"class":"yvr-flights__label yvr-flights__scheduled-label yvr-flights__scheduled-label--arrived notranslate"}))



51

In [286]:
len(soup.findAll('td',{'class':"id"}))

0

In [295]:
len(soup.tbody.findAll('tr', {"class":"yvr-flights__row yvr-flights__row--earlier yvr-flights__row--hidden yvr-flights__row--arrived"}))

47

In [231]:
len(soup.findAll(['div','time'], {"class": ["yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--arrived",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--delayed",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--cancelled",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--early",
                                           "yvr-flights__label yvr-flights__scheduled-label yvr-flights__scheduled-label--str_on_time notranslate"]}))

87

In [273]:
soup.find_all(['div','time'], {"class": ["yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--arrived",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--delayed",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--cancelled",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--early",
                                           "yvr-flights__label yvr-flights__scheduled-label yvr-flights__scheduled-label--str_on_time notranslate"]}).get_text()


AttributeError: ResultSet object has no attribute 'get_text'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [ ]:
soup.findAll(['div','time'], {"class": ["yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--arrived",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--delayed",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--cancelled",
                                           "yvr-flights__label yvr-flights__revised-label yvr-flights__revised-label--early",
                                           "yvr-flights__label yvr-flights__scheduled-label yvr-flights__scheduled-label--str_on_time notranslate"]})


In [238]:
len(soup.findAll(['div', 'time'], {'class': ["yvr-flights__label yvr-flights__scheduled-label yvr-flights__scheduled-label--arrived notranslate",
                                            "",
                                            "",
                                            "",
                                            ""]}))
    
    

51